In [1]:
import pickle 

with open('../../data/Hazumi_features/Hazumi1911_features.pkl', mode='rb') as f:
    SS, TS, SP, TP, text, audio, visual, bio, vid = pickle.load(f, encoding='utf-8')

In [9]:
for id in vid:
    count = 0
    for ss, ts in zip(SS[id], TS[id]):
        if ss == 0 and (ts == 1 or ts == 2):
            count += 1
        elif ts == 0 and (ss == 1 or ss == 2):
            count += 1 
    print(round(count/len(SS[id]),2))

0.24
0.39
0.27
0.51
0.56
0.23
0.28
0.07
0.06
0.36
0.36
0.02
0.01
0.02
0.14
0.33
0.2
0.13
0.08
0.06
0.01
0.14
0.06
0.37
0.22
0.08


In [11]:
for id in vid:
    count = 0
    for ss, ts in zip(SS[id], TS[id]):
        if ss != ts:
            count += 1
    print(id, round(count/len(SS[id]),2))

1911F2001 0.42
1911F2002 0.78
1911F3001 0.53
1911F3002 0.73
1911F3003 0.71
1911F4001 0.65
1911F4002 0.62
1911F4003 0.26
1911F5001 0.42
1911F5002 0.64
1911F6001 0.69
1911F6002 0.33
1911F6003 0.3
1911F7002 0.39
1911M2001 0.41
1911M2002 0.45
1911M2003 0.56
1911M4001 0.44
1911M4002 0.3
1911M5001 0.35
1911M5002 0.13
1911M6001 0.35
1911M6002 0.24
1911M6003 0.51
1911M7001 0.48
1911M7002 0.32
